# Same as feature_engineering in /notebooks folder

In [77]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate

In [78]:
df = pd.read_csv('../validation_data/validation_data.csv')
df = df.drop('ptnum', axis=1)
df.drop(columns=['C-20565-8'], inplace=True)
df_codes = pd.read_csv('../data/codes.csv')
code_to_name = df_codes.set_index('code')['name'].to_dict()
df = df.rename(columns=code_to_name)
df2 = df.copy()
df.head()

,label,scc,race,marital,ethnic,gender,state,age,Pain severity - 0-10 verbal numeric rating [Score] - Reported,Influenza seasonal injectable preservative free,...,Parainfluenza virus 1 RNA [Presence] in Respiratory specimen by NAA with probe detection,Influenza virus B RNA [Presence] in Respiratory specimen by NAA with probe detection,Influenza virus A RNA [Presence] in Respiratory specimen by NAA with probe detection,Adenovirus A+B+C+D+E DNA [Presence] in Respiratory specimen by NAA with probe detection,SARS-CoV-2 RNA Pnl Resp NAA+probe,Hydroxychloroquine Sulfate 200 MG Oral Tablet,1 ML denosumab 60 MG/ML Prefilled Syringe,Fexofenadine hydrochloride 60 MG Oral Tablet,Leronlimab 700 MG Injection,Lenzilumab 200 MG IV
0,0,113,white,m,hispanic,m,massachusetts,50t70,abnormal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,100,white,s,nonhispanic,f,massachusetts,50t70,abnormal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,106,other,m,nonhispanic,f,massachusetts,30t50,normal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,74,hawaiian,m,nonhispanic,m,massachusetts,gt70,abnormal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,112,white,m,nonhispanic,f,massachusetts,50t70,abnormal,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
X = pd.read_csv('../transformed_data/X_train.csv')
train_cols = X.columns.to_list()

In [80]:
columns = df.columns
unique_columns = list(set(columns))  # Get unique column names

# Create a new DataFrame with combined columns
df_combined = pd.DataFrame()

# Iterate over the unique columns to combine and merge appropriately
for col in unique_columns:
    if list(columns).count(col) > 1:  # If the column name is duplicated
        # Find all columns with this name and combine them, e.g., with first non-null
        combined_series = df[[col]].bfill(axis=1).iloc[:, 0]  # Backfill to combine
        df_combined[col] = combined_series
    else:
        df_combined[col] = df[col]  # If it's unique, just copy the column

print("\nDataFrame after combining columns with the same name:")
print(df_combined)


DataFrame after combining columns with the same name:
     Memantine hydrochloride 2 MG/ML Oral Solution Pneumonia (disorder)  \
0                                              NaN                  NaN   
1                                              NaN                  NaN   
2                                              NaN                  NaN   
3                                              NaN                  NaN   
4                                              NaN                  NaN   
...                                            ...                  ...   
3874                                           NaN                  NaN   
3875                                           NaN                  NaN   
3876                                           NaN                  NaN   
3877                                           NaN                  NaN   
3878                                           NaN                  NaN   

     Coronary artery bypass grafting  \
0   

In [81]:
df2 = df_combined.copy()

### Creating new features

In [82]:
category_dict = {}

In [83]:
respiratory_disorders_columns = [
    'Acute bacterial sinusitis (disorder)',
    'Acute bronchitis (disorder)',
    'Acute respiratory distress syndrome (disorder)',
    'Acute viral pharyngitis (disorder)',
    'Asthma',
    'Asthma screening',
    'Bacterial infectious disease (disorder)',
    'COVID-19',
    'Chronic obstructive bronchitis (disorder)',
    'Cough (finding)',
    'Dyspnea (finding)',
    'Fever (finding)',
    'Hemoptysis (finding)',
    'Hypoxemia (disorder)',
    'Influenza virus A Ag [Presence] in Nasopharynx by Rapid immunoassay',
    'Influenza virus B Ag [Presence] in Nasopharynx by Rapid immunoassay',
    'Measurement of respiratory function (procedure)',
    'NDA020503 200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Dose Inhaler',
    'Nasal congestion (finding)',
    'Nasal sinus endoscopy (procedure)',
    'Otitis media',
    'Oxygen Therapy',
    'Parainfluenza virus 3 RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Pneumococcal conjugate PCV 13',
    'Pneumonia (disorder)',
    'Pulmonary emphysema (disorder)',
    'Respiratory distress (finding)',
    'SARS-CoV-2 RNA Pnl Resp NAA+probe',
    'Sore throat symptom (finding)',
    'Sputum examination (procedure)',
    'Sputum finding (finding)',
    'Streptococcal sore throat (disorder)',
    'Streptococcus pneumoniae group B antigen test',
    'Throat culture (procedure)',
    'Total knee replacement',
    'Transfer to stepdown unit (procedure)',
    'Transplant of lung (procedure)',
    'Viral sinusitis (disorder)',
    'House dust mite (organism)',
    'Mold (organism)',
    'Grass pollen (substance)',
    'Tree pollen (substance)',
    'Animal dander (substance)',
    'Wheezing (finding)'
]


df2['Respiratory Disorders'] = df2[respiratory_disorders_columns].sum(axis=1)
category_dict['Respiratory Disorders'] = respiratory_disorders_columns

In [84]:
heart_and_cardiovascular_diseases_columns = [
    'Acute deep venous thrombosis (disorder)',
    'Acute pulmonary embolism (disorder)',
    'Atrial Fibrillation',
    'Blindness due to type 2 diabetes mellitus (disorder)',
    'Chronic congestive heart failure (disorder)',
    'Coronary Heart Disease',
    'Diabetes',
    'Diabetic renal disease (disorder)',
    'Diabetic retinopathy associated with type II diabetes mellitus (disorder)',
    'Hyperglycemia (disorder)',
    'Hypertension',
    'Hyperlipidemia',
    'Hypertriglyceridemia (disorder)',
    'Myocardial Infarction',
    'Stroke',
    'Heart failure (disorder)'
]


df2['Heart and Cardiovascular Diseases'] = df2[heart_and_cardiovascular_diseases_columns].sum(axis=1)
category_dict['Heart and Cardiovascular Diseases'] = heart_and_cardiovascular_diseases_columns

In [85]:
metabolic_and_endocrine_disorders_columns = [
    'Alcoholism',
    'Anemia (disorder)',
    'Body mass index 30+ - obesity (finding)',
    'Chronic kidney disease stage 1 (disorder)',
    'Chronic kidney disease stage 2 (disorder)',
    'Chronic kidney disease stage 3 (disorder)',
    'Fatigue (finding)',
    'Gout',
    'Hyperlipidemia',
    'Hypertension',
    'Hypertriglyceridemia (disorder)',
    'Hypoxemia (disorder)',
    'Idiopathic atrophic hypothyroidism',
    'Loss of taste (finding)',
    'Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)',
    'Metabolic syndrome X (disorder)',
    'Microalbuminuria due to type 2 diabetes mellitus (disorder)',
    'Neuropathy due to type 2 diabetes mellitus (disorder)',
    'Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)',
    'Osteoarthritis of hip',
    'Osteoarthritis of knee',
    'Osteoporosis (disorder)',
    'Pathological fracture due to osteoporosis (disorder)',
    'Prediabetes',
    'Proteinuria due to type 2 diabetes mellitus (disorder)',
    'Protracted diarrhea',
    'Rheumatoid arthritis',
    'Secondary malignant neoplasm of colon',
    'Stress (finding)',
    'Unhealthy alcohol drinking behavior (finding)',
    'Whiplash injury to neck',
    'Proliferative diabetic retinopathy due to type II diabetes mellitus (disorder)'
]

df2['Metabolic and Endocrine Disorders'] = df2[metabolic_and_endocrine_disorders_columns].sum(axis=1)
category_dict['Metabolic and Endocrine Disorders'] = metabolic_and_endocrine_disorders_columns

In [86]:
neurological_disorders_columns = [
    'Alzheimer\'s disease (disorder)',
    'Brain damage - traumatic',
    'Chronic intractable migraine without aura',
    'Chronic low back pain (finding)',
    'Chronic neck pain (finding)',
    'Chronic pain',
    'Chronic paralysis due to lesion of spinal cord',
    'Chronic sinusitis (disorder)',
    'Epilepsy',
    'Familial Alzheimer\'s disease of early onset (disorder)',
    'Fibromyalgia (disorder)',
    'Lupus erythematosus',
    'Male infertility due to cystic fibrosis (disorder)',
    'Malignant neoplasm of breast (disorder)',
    'Malignant tumor of colon',
    'Seizure disorder',
    'Sepsis (disorder)',
    'Sepsis caused by virus (disorder)',
    'Septic shock (disorder)',
    'Shock (disorder)',
    'Sinusitis (disorder)',
    'Streptococcal sore throat (disorder)',
    'Suspected COVID-19',
    'Transformed migraine (disorder)',
    'Victim of intimate partner abuse (finding)',
    'Viral sinusitis (disorder)',
    'Seizure Count Cerebral Cortex Electroencephalogram (EEG)',
    'Headache (finding)',
    'Primary fibromyalgia syndrome'
]


df2['Neurological Disorders'] = df2[neurological_disorders_columns].sum(axis=1)
category_dict['Neurological Disorders'] = neurological_disorders_columns

In [87]:
orthopedic_injuries_columns = [
    'Admission to orthopedic department',
    'Ankle X-ray',
    'Bone immobilization',
    'Brain damage - traumatic',
    'Bullet wound',
    'Burn injury(morphologic abnormality)',
    'Chronic kidney disease stage 1 (disorder)',
    'Clavicle X-ray',
    'Closed fracture of hip',
    'Concussion injury of brain',
    'Concussion with loss of consciousness',
    'Concussion with no loss of consciousness',
    'Facial laceration',
    'Fracture of ankle',
    'Fracture of clavicle',
    'Fracture of forearm',
    'Fracture of rib',
    'Fracture of the vertebral column with spinal cord injury',
    'Fracture of vertebral column without spinal cord injury',
    'Fracture subluxation of wrist',
    'History of amputation of foot (situation)',
    'History of lower limb amputation (situation)',
    'Hypertriglyceridemia (disorder)',
    'Impacted molars',
    'Injury of anterior cruciate ligament',
    'Injury of heart (disorder)',
    'Injury of kidney (disorder)',
    'Injury of medial collateral ligament of knee',
    'Injury of tendon of the rotator cuff of shoulder',
    'Joint pain (finding)',
    'Laceration of foot',
    'Laceration of hand',
    'Laceration of thigh',
    'Localized  primary osteoarthritis of the hand',
    'Muscle pain (finding)',
    'Rupture of appendix',
    'Rupture of patellar tendon',
    'Second degree burn',
    'Sprain of ankle',
    'Sprain of wrist',
    'Surgical manipulation of joint of knee',
    'Surgical manipulation of shoulder joint',
    'Tear of meniscus of knee',
    'Total knee replacement',
    'Whiplash injury to neck',
    'Third degree burn'
]


df2["Orthopedic Injuries"] = df2[orthopedic_injuries_columns].sum(axis=1)
category_dict['Orthopedic Injuries'] = orthopedic_injuries_columns

In [88]:
mental_health_columns = [
    'Alcoholism',
    'At risk for suicide (finding)',
    'Brief general examination (procedure)',
    'Care regimes assessment (procedure)',
    'Comprehensive interview and evaluation (procedure)',
    'Concussion with no loss of consciousness',
    'Depression screening (procedure)',
    'Depression screening using Patient Health Questionnaire Nine Item score (procedure)',
    'Depression screening using Patient Health Questionnaire Two-Item score (procedure)',
    'Died in hospice (finding)',
    'Drug overdose',
    'Evaluation of psychiatric state of patient',
    'Has a criminal record (finding)',
    'Homeless (finding)',
    'Limited social contact (finding)',
    'Mental health Outpatient Note',
    'Mental health Telehealth Note',
    'Mental health screening (procedure)',
    'Misuses drugs (finding)',
    'Nausea (finding)',
    'Not in labor force (finding)',
    'Part-time employment (finding)',
    'Passive conjunctival congestion (finding)',
    'Posttraumatic stress disorder',
    'Psychiatric follow-up',
    'Refugee (person)',
    'Reports of violence in the environment (finding)',
    'Severe anxiety (panic) (finding',
    'Social isolation (finding)',
    'Stress (finding)',
    'Suicide risk assessment (procedure)',
    'Victim of intimate partner abuse (finding)',
    'Attempted suicide - cut/stab',
    'Attempted suicide - suffocation',
    'Opioid abuse (disorder)',
    'Assessment of anxiety (procedure)'
]


df2['Mental Health'] = df2[mental_health_columns].sum(axis=1)
category_dict['Mental Health'] = mental_health_columns

In [89]:
reproductive_and_pregancy_columns = [
    '1 ML medroxyPROGESTERone acetate 150 MG/ML Injection',
    '168 HR Ethinyl Estradiol 0.00146 MG/HR / norelgestromin 0.00625 MG/HR Transdermal System',
    'Antenatal RhD antibody screening',
    'Antepartum eclampsia',
    'Augmentation of labor',
    'Auscultation of the fetal heart',
    'Bilateral tubal ligation',
    'Blighted ovum',
    'Camila 28 Day Pack',
    'Counseling for termination of pregnancy',
    'Cytopathology procedure  preparation of smear  genital source',
    'Episiotomy',
    'Errin 28 Day Pack',
    'Estrostep Fe 28 Day Pack',
    'Etonogestrel 68 MG Drug Implant',
    'Evaluation of uterine fundal height',
    'Excision of fallopian tube and surgical removal of ectopic pregnancy',
    'Fetal anatomy study',
    'Fetus with unknown complication',
    'Gonorrhea infection test',
    'Hyperlipidemia',
    'Induced termination of pregnancy',
    'Insertion of intrauterine contraceptive device',
    'Insertion of subcutaneous contraceptive',
    'Instrumental delivery',
    'Jolivette 28 Day Pack',
    'Kyleena 19.5 MG Intrauterine System',
    'Leronlimab 700 MG Injection',
    'Levonorgestrel 0.00354 MG/HR Drug Implant',
    'Levora 0.15/30 28 Day Pack',
    'Liletta 52 MG Intrauterine System',
    'Medical induction of labor',
    'Mestranol / Norethynodrel [Enovid]',
    'Methotrexate injection into tubal pregnancy',
    'Mirena 52 MG Intrauterine System',
    'Miscarriage in first trimester',
    'Natazia 28 Day Pack',
    'Norinyl 1+50 28 Day Pack',
    'NuvaRing 0.12/0.015 MG per 24HR 21 Day Vaginal Ring',
    'Ortho Tri-Cyclen 28 Day Pack',
    'Preeclampsia',
    'Pregnancy termination care',
    'Premature birth of newborn',
    'Removal of intrauterine device',
    'Removal of subcutaneous contraceptive',
    'Replacement of contraceptive intrauterine device',
    'RhD passive immunization',
    'Screening for chromosomal aneuploidy in prenatal amniotic fluid',
    'Spontaneous breech delivery',
    'Standard pregnancy test',
    'Tubal pregnancy',
    'Ultrasonography of abdomen  right upper quadrant and epigastrium',
    'Ultrasonography of bilateral breasts (procedure)',
    'Ultrasound scan for fetal viability',
    'Vaccination for diphtheria  pertussis  and tetanus',
    'Vasectomy' 
]

df2['Reproductive and Pregnancy'] = df2[reproductive_and_pregancy_columns].sum(axis=1)
category_dict['Reproductive and Pregnancy'] = reproductive_and_pregancy_columns

In [90]:
pain_relievers_and_analesics_columns = [
    '10 ML Alfentanil 0.5 MG/ML Injection',
    '10 ML Fentanyl 0.05 MG/ML Injection',
    '12 HR Hydrocodone Bitartrate 10 MG Extended Release Oral Capsule',
    '5 ML SUFentanil 0.05 MG/ML Injection',
    '72 HR Fentanyl 0.025 MG/HR Transdermal System',
    'Abuse-Deterrent 12 HR Oxycodone Hydrochloride 10 MG Extended Release Oral Tablet [Oxycontin]',
    'Abuse-Deterrent 12 HR Oxycodone Hydrochloride 15 MG Extended Release Oral Tablet',
    'Acetaminophen 325 MG / oxyCODONE Hydrochloride 2.5 MG Oral Tablet',
    'Acetaminophen 325 MG / oxyCODONE Hydrochloride 5 MG Oral Tablet',
    'Acetaminophen 500 MG Oral Tablet',
    'Acetaminophen 300 MG / Hydrocodone Bitartrate 5 MG Oral Tablet',
    'Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet',
    'Acetaminophen 325 MG / Oxycodone Hydrochloride 10 MG Oral Tablet [Percocet]',
    'Acetaminophen 325 MG Oral Tablet',
    'Acetaminophen/Hydrocodone',
    'Aspirin',
    'Aspirin 81 MG Oral Tablet',
    'Carbamazepine[Tegretol]',
    'Chlorpheniramine Maleate 4 MG Oral Tablet',
    'Clopidogrel 75 MG Oral Tablet',
    'Colchicine 0.6 MG Oral Tablet',
    'Cyclophosphamide 1000 MG Injection',
    'Diazepam 5 MG Oral Tablet',
    'Diazepam 5 MG/ML Injectable Solution',
    'Doxycycline Monohydrate 100 MG Oral Tablet',
    'Ibuprofen',
    'Ibuprofen 200 MG Oral Tablet',
    'Ibuprofen 400 MG Oral Tablet [Ibu]',
    'Lorazepam 2 MG/ML Injectable Solution',
    'Meperidine Hydrochloride 50 MG Oral Tablet',
    'Muscle pain (finding)',
    'Naproxen 500 MG Oral Tablet',
    'Naproxen sodium 220 MG Oral Tablet',
    'Nitroglycerin 0.4 MG/ACTUAT Mucosal Spray',
    'Phenazopyridine hydrochloride 100 MG Oral Tablet',
    'Sulfamethoxazole / Trimethoprim',
    'buprenorphine 2 MG / naloxone 0.5 MG Sublingual Tablet',
    'diphenhydrAMINE Hydrochloride 25 MG Oral Tablet',
    'duloxetine 20 MG Delayed Release Oral Capsule',
    'methadone hydrochloride 10 MG Oral Tablet',
    'tramadol hydrochloride 50 MG Oral Tablet',
    'clonazePAM 0.25 MG Oral Tablet',
    'Midazolam 1 MG/ML Injectable Solution',
    'Rocuronium bromide 10 MG/ML Injectable Solution',
    'Epidural anesthesia',
]


df2['Pain Relievers and Analesics'] = df2[pain_relievers_and_analesics_columns].sum(axis=1)
category_dict['Pain Relievers and Analesics'] = pain_relievers_and_analesics_columns

In [91]:
cardiovascular_and_blood_pressure_medications_columns = [
    '0.3 ML Enoxaparin sodium 100 MG/ML Prefilled Syringe',
    '0.4 ML Enoxaparin sodium 100 MG/ML Prefilled Syringe',
    '1 ML Enoxaparin sodium 150 MG/ML Prefilled Syringe',
    '1 ML heparin sodium  porcine 5000 UNT/ML Injection',
    '3 ML Amiodarone hydrocholoride 50 MG/ML Prefilled Syringe',
    '4 ML norepinephrine 1 MG/ML Injection',
    'Acute deep venous thrombosis (disorder)',
    'Acute pulmonary embolism (disorder)',
    'Alkaline phosphatase [Enzymatic activity/volume] in Serum or Plasma',
    'Alteplase 100 MG Injection',
    'Assessment using New York Heart Association Classification (procedure)',
    'Atorvastatin 80 MG Oral Tablet',
    'Atropine Sulfate 1 MG/ML Injectable Solution',
    'Captopril 25 MG Oral Tablet',
    'Carbon dioxide [Partial pressure] in Arterial blood',
    'Cardiac Arrest',
    'Cardiovascular stress testing (procedure)',
    'Catheter ablation of tissue of heart',
    'Coronary artery bypass grafting',
    'Digoxin 0.125 MG Oral Tablet',
    'Echocardiography (procedure)',
    'Electrical cardioversion',
    'Electrocardiographic procedure',
    'Furosemide 40 MG Oral Tablet',
    'Glomerular filtration rate/1.73 sq M.predicted',
    'Heart rate',
    'History of myocardial infarction (situation)',
    'Hydrochlorothiazide 25 MG Oral Tablet',
    'Implantation of left ventricular assist device (procedure)',
    'Insertion of biventricular implantable cardioverter defibrillator',
    'Lisinopril',
    'Low Density Lipoprotein Cholesterol',
    'Myocardial Infarction',
    'Nitrofurantoin 5 MG/ML Oral Suspension',
    'Peripheral blood smear interpretation',
    'Referral to hypertension clinic',
    'Shock (disorder)',
    'Stroke',
    'Transplantation of heart (procedure)',
    'Verapamil Hydrochloride 40 MG',
    'Warfarin Sodium 5 MG Oral Tablet',
    'carvedilol 25 MG Oral Tablet',
    'lisinopril 10 MG Oral Tablet',
    'lisinopril 20 MG Oral Tablet',
    'losartan potassium 25 MG Oral Tablet',
    'losartan potassium 50 MG Oral Tablet',
    'sacubitril 97 MG / valsartan 103 MG Oral Tablet',
    'remifentanil 2 MG Injection',
    'pregabalin 100 MG Oral Capsule'
]


df2['Cardiovascular and Blood Pressure'] = df2[cardiovascular_and_blood_pressure_medications_columns].sum(axis=1)
category_dict['Cardiovascular and Blood Pressure'] = cardiovascular_and_blood_pressure_medications_columns

In [92]:
injection_medications_columns = [
    '10 ML Doxorubicin Hydrochloride 2 MG/ML Injection',
    '10 ML oxaliplatin 5 MG/ML Injection',
    '1 ML DOCEtaxel 20 MG/ML Injection',
    '1 ML Epinephrine 1 MG/ML Injection',
    '1 ML Morphine Sulfate 5 MG/ML Injection',
    '1 ML Vasopressin (USP) 20 UNT/ML Injection',
    '10 ML Fluorouracil 50 MG/ML Injection',
    '10 ML Pamidronate Disodium 3 MG/ML Injection',
    '100 ML Epirubicin Hydrochloride 2 MG/ML Injection',
    '100 ML Propofol 10 MG/ML Injection',
    '100 ML zoledronic acid 0.04 MG/ML Injection',
    '150 ML vancomycin 5 MG/ML Injection',
    '2 ML Ondansetron 2 MG/ML Injection',
    '20 ML tocilizumab 20 MG/ML Injection',
    '5 ML hyaluronidase-oysk 2000 UNT/ML / trastuzumab 120 MG/ML Injection',
    'pneumococcal polysaccharide vaccine  23 valent',
    'remdesivir 100 MG Injection',
    'zoster',
    'Aztreonam 2000 MG Injection',
    'cefdinir', 
    'Cefdinir',
    'doxycycline hyclate 100 MG',
    'Ampicillin 100 MG/ML Injectable Solution',
    'Penicillin G 375 MG/ML Injectable Solution',
    'Penicillin V',
    'Paclitaxel 100 MG Injection',
    'Piperacillin 4000 MG / tazobactam 500 MG Injection',
    'Leucovorin 100 MG Injection',
    'Influenza  seasonal  injectable  preservative free',
    'Syphilis infection test',
    'Skin test for tuberculosis',
    'Urine culture'
]


df2['Injection Medications'] = df2[injection_medications_columns].sum(axis=1)
category_dict['Injection Medications'] = injection_medications_columns

In [93]:
oral_medications_columns = [
    '12 HR Cefaclor 500 MG Extended Release Oral Tablet',
    '24 HR Donepezil hydrochloride 10 MG / Memantine hydrochloride 28 MG Extended Release Oral Capsule',
    '24 HR Metformin hydrochloride 500 MG Extended Release Oral Tablet',
    'Acetaminophen 300 MG / Codeine Phosphate 15 MG Oral Tablet',
    'Acetaminophen 325 MG Oral Tablet [Tylenol]',
    'Alendronic acid 10 MG Oral Tablet',
    'Allopurinol 100 MG Oral Tablet',
    'Amlodipine 5 MG Oral Tablet',
    'Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet',
    'Amoxicillin 250 MG Oral Capsule',
    'Aspirin 81 MG Oral Tablet',
    'Astemizole 10 MG Oral Tablet',
    'Atorvastatin 80 MG Oral Tablet',
    'Azithromycin 250 MG Oral Tablet',
    'Azithromycin 250mg',
    'Cefuroxime 250 MG Oral Tablet',
    'Chlorpheniramine Maleate 4 MG Oral Tablet',
    'Clopidogrel 75 MG Oral Tablet',
    'Colchicine 0.6 MG Oral Tablet',
    'Diazepam 5 MG Oral Tablet',
    'Digoxin 0.125 MG Oral Tablet',
    'Donepezil hydrochloride 10 MG Oral Tablet',
    'Donepezil hydrochloride 23 MG Oral Tablet',
    'Doxycycline Monohydrate 100 MG Oral Tablet',
    'Fexofenadine hydrochloride 60 MG Oral Tablet',
    'Furosemide 40 MG Oral Tablet',
    'Galantamine 4 MG Oral Tablet',
    'Hydrochlorothiazide 25 MG Oral Tablet',
    'Levothyroxine Sodium 0.075 MG Oral Tablet',
    'Loratadine 10 MG Oral Tablet',
    'Methotrexate 2.5 MG Oral Tablet',
    'Milnacipran hydrochloride 100 MG Oral Tablet',
    'Nitrofurantoin 5 MG/ML Oral Suspension',
    'Penicillin V Potassium 500 MG Oral Tablet',
    'Phenazopyridine hydrochloride 100 MG Oral Tablet',
    'Sertraline 100 MG Oral Tablet',
    'Simvastatin 10 MG Oral Tablet',
    'Simvastatin 20 MG Oral Tablet',
    'Tacrine 10 MG Oral Capsule',
    'Tamoxifen 10 MG Oral Tablet',
    'Terfenadine 60 MG Oral Tablet',
    'Verapamil Hydrochloride 40 MG',
    'Verzenio 100 MG Oral Tablet',
    'Warfarin Sodium 5 MG Oral Tablet',
    'palbociclib 100 MG Oral Capsule',
    'predniSONE 20 MG Oral Tablet',
    'ribociclib 200 MG Oral Tablet',
    'neratinib 40 MG Oral Tablet',
    'Hydroxychloroquine Sulfate 200 MG Oral Tablet',
    'chloroquine phosphate 500 MG Oral Tablet',
    'Naltrexone hydrochloride 50 MG Oral Tablet',
    'lapatinib 250 MG Oral Tablet',
    'cetirizine hydrochloride 10 MG Oral Tablet',
    'cycloSPORINE  modified 100 MG Oral Capsule',
    'letrozole 2.5 MG Oral Tablet',
    'exemestane 25 MG Oral Tablet',
    'ferrous sulfate 325 MG Oral Tablet'
    
]


df2['Oral Medications'] = df2[oral_medications_columns].sum(axis=1)
category_dict['Oral Medications'] = oral_medications_columns

In [94]:
other_medications_columns = [
    '0.25 ML Leuprolide Acetate 30 MG/ML Prefilled Syringe',
    '1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen]',
    '1 ML denosumab 60 MG/ML Prefilled Syringe',
    '10 ML Furosemide 10 MG/ML Injection',
    '3 ML liraglutide 6 MG/ML Pen Injector',
    '5 ML fulvestrant 50 MG/ML Prefilled Syringe',
    '12 HR Cefaclor 500 MG Extended Release Oral Tablet',
    '120 ACTUAT Fluticasone propionate 0.044 MG/ACTUAT Metered Dose Inhaler',
    '60 ACTUAT Fluticasone propionate 0.25 MG/ACTUAT / salmeterol 0.05 MG/ACTUAT Dry Powder Inhaler',
    'Acetaminophen 21.7 MG/ML / Dextromethorphan Hydrobromide 1 MG/ML / doxylamine succinate 0.417 MG/ML Oral Solution',
    'Chlamydia antigen test',
    'Hepatitis B Surface Antigen Measurement',
    'Hepatitis C antibody test',
    'Human immunodeficiency virus antigen test',
    'Insulin Lispro 100 UNT/ML Injectable Solution [Humalog]',
    'Jolivette 28 Day Pack',
    'Lymphocytes [#/volume] in Blood by Automated count',
    'Measurement of Varicella-zoster virus antibody',
    'Memantine hydrochloride 2 MG/ML Oral Solution',
    'NDA020503 200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Dose Inhaler',
    'NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector',
    'NITROFURANTOIN  MACROCRYSTALS 50 MG Oral Capsule',
    'Ortho Tri-Cyclen 28 Day Pack',
    'Pneumococcal conjugate PCV 13',
    'Pulmozyme (Dornase Alfa)',
    'SARS-COV-2 (COVID-19) vaccine  mRNA  spike protein  LNP  preservative free  100 mcg/0.5mL dose',
    'SARS-COV-2 (COVID-19) vaccine  mRNA  spike protein  LNP  preservative free  30 mcg/0.3mL dose',
    'SARS-COV-2 (COVID-19) vaccine  vector non-replicating  recombinant spike protein-Ad26  preservative free  0.5 mL',
    'Seasonique 91 Day Pack',
    'Td (adult) preservative free',
    'Trinessa 28 Day Pack',
    'Vaccination for diphtheria  pertussis  and tetanus',
    'Vitamin B 12 5 MG/ML Injectable Solution',
    'Vomiting symptom (finding)',
    'Yaz 28 Day Pack',
    'ado-trastuzumab emtansine 100 MG Injection',
    'albuterol 5 MG/ML Inhalation Solution',
    'amLODIPine 2.5 MG Oral Tablet',
    'anastrozole 1 MG Oral Tablet',
    'baricitinib 2 MG Oral Tablet',
    'buprenorphine 2 MG / naloxone 0.5 MG Sublingual Tablet',
    'carvedilol 25 MG Oral Tablet',
    'sevoflurane 1000 MG/ML Inhalant Solution',
    'Latex (substance)',
    'desflurane 1000 MG/ML Inhalation Solution',
    'Isoflurane 999 MG/ML Inhalant Solution',
    'Hydrocortisone 10 MG/ML Topical Cream',
    'Lenzilumab 200 MG IV',
    'remifentanil 2 MG Injection',
    'Carboplatin 10 MG/ML Injectable Solution',
    'chloroquine phosphate 500 MG Oral Tablet',
    'Naltrexone hydrochloride 50 MG Oral Tablet',
    'lapatinib 250 MG Oral Tablet',
    'Sodium Chloride 9 MG/ML Injectable Solution',
    'insulin human  isophane 70 UNT/ML / Regular Insulin  Human 30 UNT/ML Injectable Suspension [Humulin]'
]


df2['Other Medications'] = df2[other_medications_columns].sum(axis=1)
category_dict['Other Medications'] = other_medications_columns

In [95]:
therapies_and_regimes_columns = [
    '24hr nicotine transdermal patch',
    'Assessment of health and social care needs (procedure)',
    'Assessment of substance use (procedure)',
    'Assessment using Alcohol Use Disorders Identification Test - Consumption (procedure)',
    'Assessment using Morse Fall Scale (procedure)',
    'Cognitive and behavioral therapy (regime/therapy)',
    'Combined chemotherapy and radiation therapy (procedure)',
    'Comprehensive interview and evaluation (procedure)',
    'Construction of diverting colostomy',
    'Controlled ventilation procedure and therapy  initiation and management (procedure)',
    'Coordination of care plan (procedure)',
    'Home health aide service (regime/therapy)',
    'Hospice care (regime/therapy)',
    'Interstitial brachytherapy (procedure)',
    'Intracavitary brachytherapy (procedure)',
    'Monitoring of patient (regime/therapy)',
    'Movement therapy (regime/therapy)',
    'Nursing care/supplementary surveillance (regime/therapy)',
    'Occupational therapy (regime/therapy)',
    'Physical examination',
    'Physical therapy procedure (regime/therapy)',
    'Professional / ancillary services care (regime/therapy)',
    'Psychosocial care (regime/therapy)',
    'Pulmonary rehabilitation (regime/therapy)',
    'Radiation oncology AND/OR radiotherapy (procedure)',
    'Radiation therapy care (regime/therapy)',
    'Referral to home health care service (procedure)',
    'Referral to hypertension clinic',
    'Social case work (regime/therapy)',
    'Speech and language therapy regime (regime/therapy',
    'Subcutaneous immunotherapy',
    'Teleradiotherapy procedure (procedure)',
    'Transplant of lung (procedure)',
    'Transplantation of heart (procedure)',
    'Weaning from mechanically assisted ventilation (procedure)',
    'Microbial culture (procedure)',
    'Chemotherapy (procedure)'
]


df2['Therapies and Regimes'] = df2[therapies_and_regimes_columns].sum(axis=1)
category_dict['Therapies and Regimes'] = therapies_and_regimes_columns

In [96]:
diagnostic_procedures_columns = [
    'Biopsy of breast (procedure)',
    'Biopsy of colon',
    'Biopsy of prostate',
    'Blood typing  RH typing',
    'Bone density scan (procedure)',
    'Chlamydia antigen test',
    'Clavicle X-ray',
    'Colonoscopy',
    'Digital examination of rectum',
    'Echocardiography (procedure)',
    'Electrocardiographic procedure',
    'Human epidermal growth factor receptor 2 gene detection by fluorescence in situ hybridization (procedure)',
    'Human epidermal growth factor receptor 2 gene detection by immunohistochemistry (procedure)',
    'Human immunodeficiency virus antigen test',
    'Initial patient assessment (procedure)',
    'Knee X-ray',
    'Magnetic resonance imaging of breast (procedure)',
    'Mammogram - symptomatic (procedure)',
    'Mammography (procedure)',
    'Manual pelvic examination (procedure)',
    'Measurement of Varicella-zoster virus antibody',
    'Measurement of respiratory function (procedure)',
    'Nasal sinus endoscopy (procedure)',
    'Pelvis X-ray',
    'Plain chest X-ray (procedure)',
    'Screening for domestic abuse (procedure)',
    'Screening for drug abuse (procedure)',
    'Screening for occult blood in feces (procedure)',
    'Screening mammography (procedure)',
    'Sentinel lymph node biopsy (procedure)',
    'Spirometry (procedure)',
    'Throat culture (procedure)',
    'Upper arm X-ray',
    'X-ray or wrist',
    'Seizure Count Cerebral Cortex Electroencephalogram (EEG)',
    'Allergy screening test',
    'Alpha-fetoprotein test',
    'Bilirubin.total [Presence] in Urine by Test strip',
    'Urine screening for glucose',
    'Urine screening test for diabetes',
    'Urine protein test',
    'Hemoglobin / Hematocrit / Platelet count',
    'Assessment of anxiety (procedure)',
    'Urine culture'
]


df2['Diagnostic Procedures'] = df2[diagnostic_procedures_columns].sum(axis=1)
category_dict['Diagnostic Procedures'] = diagnostic_procedures_columns

In [97]:
surgerical_interventions_columns = [
    'Admission to burn unit',
    'Admission to long stay hospital',
    'Admission to neurosurgical department',
    'Admission to trauma surgery department',
    'Admit to ICU (procedure)',
    'Amputation of right foot',
    'Amputation of right leg',
    'Appendectomy',
    'Artificial respiration (procedure)',
    'Brachytherapy of breast (procedure)',
    'Catheter ablation of tissue of heart',
    'Coronary artery bypass grafting',
    'Excision of axillary lymph node (procedure)',
    'Excision of breast tissue (procedure)',
    'Excision of sentinel lymph node (procedure)',
    'Extraction of wisdom tooth',
    'Implantation of left ventricular assist device (procedure)',
    'Induced termination of pregnancy',
    'Insertion of biventricular implantable cardioverter defibrillator',
    'Insertion of endotracheal tube (procedure)',
    'Instrumental delivery',
    'Intubation',
    'Laparoscopic Removal of Gall Bladder',
    'Lumpectomy of breast (procedure)',
    'Lung volume reduction surgery (procedure)',
    'Open Removal of Gall Bladder',
    'Partial resection of colon',
    'Percutaneous coronary intervention',
    'Percutaneous mechanical thrombectomy of portal vein using fluoroscopic guidance',
    'Prostatectomy',
    'Rectal polypectomy',
    'Removal of endotracheal tube (procedure)',
    'Removal of subcutaneous contraceptive',
    'Surgical manipulation of joint of knee',
    'Surgical manipulation of shoulder joint',
    'Suture open wound',
    'Tear of meniscus of knee',
    'Transplantation of heart (procedure)'
]

df2['Surgical Interventions'] = df2[surgerical_interventions_columns].sum(axis=1)
category_dict['Surgical Interventions'] = surgerical_interventions_columns

In [98]:
patient_care_management_columns = [
    'Assessment of health and social care needs (procedure)',
    'Bleeding from anus',
    'Bullet wound',
    'Chronic low back pain (finding)',
    'Contact dermatitis',
    'Cystitis',
    'Diarrhea symptom (finding)',
    'Escherichia coli urinary tract infection',
    'Evaluation of psychiatric state of patient',
    'Facial laceration',
    'First degree burn',
    'Heart failure education (procedure)',
    'Hemodialysis (procedure)',
    'Hep A  adult',
    'History AND physical examination (procedure)',
    'History of amputation of foot (situation)',
    'History of appendectomy',
    'History of cardiac arrest (situation)',
    'History of lower limb amputation (situation)',
    'History of single seizure (situation)',
    'Hospital admission  short-term  24 hours',
    'Information gathering (procedure)',
    'Injection of tetanus antitoxin',
    'Intramuscular injection',
    'Intravenous blood transfusion of packed cells (procedure)',
    'Intravenous injection (procedure)',
    'Laceration of foot',
    'Laceration of forearm',
    'Laceration of hand',
    'Laceration of thigh',
    'Medication Reconciliation (procedure)',
    'Movement therapy (regime/therapy)',
    'Notifications (procedure)',
    'Oxygen administration by mask (procedure)',
    'Patient discharge (procedure)',
    'Placing subject in prone position (procedure)',
    'Postoperative procedure education (procedure)',
    'Pre-discharge assessment (procedure)',
    'Radiation oncology AND/OR radiotherapy (procedure)',
    'Referral to home health care service (procedure)',
    'Referral to hypertension clinic',
    'Renal dialysis (procedure)',
    'Resuscitation using intravenous fluid (procedure)',
    'Review of systems (procedure)',
    'Rubella screening',
    'Screening mammography (procedure)',
    'Subcutaneous immunotherapy',
    'Teleradiotherapy procedure (procedure)',
    'Transfer to stepdown unit (procedure)',
    'Transfusion of plasma (procedure)',
    'Weaning from mechanically assisted ventilation (procedure)',
    'piperacillin 2000 MG / tazobactam 250 MG Injection',
    'vancomycin 1000 MG Injection',
    'Recurrent urinary tract infection',
    'Acquired coagulation disorder (disorder)',
    'Primary malignant neoplasm of colon',
    'Pyelonephritis',
    'Chill (finding)',
    'Acute Cholecystitis',
    'Cholelithiasis',
    'Appendicitis',
    'Metastasis from malignant tumor of prostate (disorder)',
    'Overlapping malignant neoplasm of colon',
    'Polyp of colon',
    'Neoplasm of prostate',
    'Carcinoma in situ of prostate (disorder)',
    'Recurrent rectal polyp',
    'Discharge from skilled nursing facility (procedure)',
    'Certification procedure (procedure)',
    'Development of individualized plan of care (procedure)'
]

df2['Patient Care Management'] = df2[patient_care_management_columns].sum(axis=1)
category_dict['Patient Care Management'] = patient_care_management_columns

In [99]:
categories = ['Respiratory Disorders', 'Heart and Cardiovascular Diseases', 'Metabolic and Endocrine Disorders', 'Neurological Disorders', 'Orthopedic Injuries', 'Mental Health', 'Reproductive and Pregnancy', 'Pain Relievers and Analesics', 'Cardiovascular and Blood Pressure', 'Injection Medications', 'Oral Medications', 'Other Medications', 'Therapies and Regimes', 'Diagnostic Procedures', 'Surgical Interventions', 'Patient Care Management']

Type of these column is object, we will convert them to int.

In [100]:
for col in df2.columns:
    # print data type of column
    print(f"Column: {col}") 
    print(f"Data Type: {df2[col].dtype}")
    if col in categories:
        # change data type to numeric
        df2[col] = pd.to_numeric(df2[col], errors='coerce')
        print(f"Data Type: {df2[col].dtype}")

Column: Memantine hydrochloride 2 MG/ML Oral Solution
Data Type: object
Column: Pneumonia (disorder)
Data Type: object
Column: Coronary artery bypass grafting
Data Type: object
Column: HER2 [Presence] in Breast cancer specimen by Immune stain
Data Type: object
Column: Full-time employment (finding)
Data Type: object
Column: Levonorgestrel 0.00354 MG/HR Drug Implant
Data Type: object
Column: Comprehensive interview and evaluation (procedure)
Data Type: object
Column: Epilepsy
Data Type: object
Column: Resuscitation using intravenous fluid (procedure)
Data Type: object
Column: SARS-COV-2 (COVID-19) vaccine  mRNA  spike protein  LNP  preservative free  30 mcg/0.3mL dose
Data Type: object
Column: Carbon Dioxide
Data Type: object
Column: NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector
Data Type: object
Column: insulin human  isophane 70 UNT/ML / Regular Insulin  Human 30 UNT/ML Injectable Suspension [Humulin]
Data Type: object
Column: Instrumental delivery
Data Type: object
Column: Prost

### Dropping unnecessary columns

Not important columns

In [101]:
unimportant_columns = ['race', 'ethnic', 'state', 'Social migrant (finding)', 'Lack of access to transportation (finding)', 'Transport problems (finding)', 'Full-time employment (finding)', 'Face mask (physical object)', 'Childbirth', 'Cesarean section', 'Normal pregnancy', 'Non-low risk pregnancy']
df2.drop(columns=unimportant_columns, inplace=True)

Columns used in new features

In [102]:
used_columns = respiratory_disorders_columns + heart_and_cardiovascular_diseases_columns + metabolic_and_endocrine_disorders_columns + neurological_disorders_columns + orthopedic_injuries_columns + mental_health_columns + reproductive_and_pregancy_columns + pain_relievers_and_analesics_columns + cardiovascular_and_blood_pressure_medications_columns + injection_medications_columns + oral_medications_columns + other_medications_columns + therapies_and_regimes_columns + diagnostic_procedures_columns + surgerical_interventions_columns + patient_care_management_columns
df2.drop(columns=used_columns, inplace=True)

Columns with only null values

### Encoding

#### Age 

In [103]:
df_one_hot = pd.get_dummies(df2['age'], prefix='age')
df2 = pd.concat([df2, df_one_hot], axis=1)
df2['age_30t50'] = df2['age_30t50'].astype(int)
df2['age_50t70'] = df2['age_50t70'].astype(int)
df2['age_gt70'] = df2['age_gt70'].astype(int)
df2.drop(columns=['age'], inplace=True)

#### Columns with normal/abnormal values

In [104]:
n_abn_columns = []
for col in df2.columns:
    if df2[col].isin(['normal', 'abnormal']).any():
        n_abn_columns.append(col)
n_abn_columns.sort()
for col in n_abn_columns:
    print(col)

Alanine aminotransferase [Enzymatic activity/volume] in Serum or Plasma
Albumin
American house dust mite IgE Ab in Serum
Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma
Bilirubin.total [Mass/volume] in Serum or Plasma
Bilirubin.total [Mass/volume] in Urine by Test strip
Body Mass Index
Body temperature
Calcium
Calcium [Mass/volume] in Serum or Plasma
Carbon Dioxide
Cat dander IgE Ab in Serum
Chloride
Chloride [Moles/volume] in Serum or Plasma
Cladosporium herbarum IgE Ab in Serum
Codfish IgE Ab in Serum
Common Ragweed IgE Ab in Serum
Cow milk IgE Ab in Serum
Creatinine
DXA [T-score] Bone density
Diastolic Blood Pressure
Egg white IgE Ab in Serum
Erythrocyte distribution width [Entitic volume] by Automated count
Erythrocyte distribution width [Ratio] by Automated count
Erythrocytes [#/volume] in Blood by Automated count
FEV1/FVC
Ferritin [Mass/volume] in Serum or Plasma
Globulin [Mass/volume] in Serum by calculation
Glucose
Glucose [Mass/volume] in Serum or Pla

Rest of them will be encoded using get_dummies.

In [105]:
df2 = pd.get_dummies(df2, columns=n_abn_columns, prefix_sep='_', dtype=int)

Other non-numeric columns will be encoded using get_dummies.

In [106]:
columns_to_encode = [
    'HER2 [Presence] in Breast cancer specimen by FISH',
    'Parainfluenza virus 1 RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Estrogen+Progesterone receptor Ag [Presence] in Tissue by Immune stain',
    'Ketones [Mass/volume] in Urine by Test strip',
    'marital',
    'Abuse Status [OMAHA]',
    'Interleukin 6 [Mass/volume] in Serum or Plasma',
    'Progesterone receptor Ag [Presence] in Breast cancer specimen by Immune stain',
    'Influenza virus A RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Parainfluenza virus 2 RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Response to cancer treatment',
    'Estrogen receptor Ag [Presence] in Breast cancer specimen by Immune stain',
    'Hemoglobin [Presence] in Urine by Test strip',
    'Housing status',
    'Influenza virus B RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Objective assessment of cardiovascular disease NYHA',
    'Drugs of abuse 5 panel - Urine by Screen method',
    'Leukocyte esterase [Presence] in Urine by Test strip',
    'Adenovirus A+B+C+D+E DNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Protein [Presence] in Urine by Test strip',
    'Appearance of Urine',
    'Capillary refill [Time] of Nail bed',
    'Treatment status Cancer',
    'Gram positive blood culture panel by Probe in Positive blood culture',
    'Glucose [Presence] in Urine by Test strip',
    'Respiratory syncytial virus RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Functional capacity NYHA',
    'Color of Urine',
    'Nitrite [Presence] in Urine by Test strip',
    'Tumor marker Cancer',
    'Tobacco smoking status NHIS',
    'gender',
    'HIV status',
    'Are you covered by health insurance or some other kind of health care plan [PhenX]',
    'Human metapneumovirus RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'Ketones [Presence] in Urine by Test strip',
    'Clarity of Urine',
    'Stage group.clinical Cancer',
    'Rhinovirus RNA [Presence] in Respiratory specimen by NAA with probe detection',
    'HER2 [Presence] in Breast cancer specimen by Immune stain',
    'Smokes tobacco daily'
]

df2 = pd.get_dummies(df2, columns=columns_to_encode, prefix_sep='_', dtype=int)

In [107]:
df2_cols = df2.columns
diff = [col for col in train_cols if col not in df2_cols]
diff

['Interleukin 6 [Mass/volume] in Serum or Plasma_5.33',
 'Ketones [Presence] in Urine by Test strip_2+']

In [108]:
df2['Interleukin 6 [Mass/volume] in Serum or Plasma_5.33'] = 0
df2['Ketones [Presence] in Urine by Test strip_2+'] = 0

In [109]:
df2 = df2.fillna(df2.median())

In [111]:
X_val = df2.drop('label', axis=1)
y_val = df2['label']

In [112]:
X_val = X_val[train_cols]

In [113]:
# save x_train and y_train to csv
X_val.to_csv('../validation_data/X_val.csv', index=False)
y_val.to_csv('../validation_data/y_val.csv', index=False)